#Stima dei Costi per il Fine-Tuning di davinci-002

In questo notebook vado  inizialmente a fare una stima di quanto costerà effettuare il fine-tuning di davinci-002 e lo confronto con i costi nel caso effettuassi il fine-tuning di gpt-3.5-turbo

In [ ]:
!pip install tiktoken
!pip install python-dotenv

In [ ]:
from google.colab import drive, files
import os
from dotenv import load_dotenv, find_dotenv
drive.mount("/content/drive/", force_remount=False)
%cd "/content/drive/My Drive/"
load_dotenv('./Lab_Python/.env')

Mounted at /content/drive/
/content/drive/My Drive


True

### Carico il dataset con il prompt e la completion, in questo caso la completion non è generata da text-davinci-003

In [ ]:
import json

with open('./Tesi/prova_domande_fine_tuning.json', 'r') as f:
    data = json.load(f)

json_str = json.dumps(data)

f.close()

json_str

Costo fine-tuning di gpt-3.5-turbo con i miei documenti

In [ ]:
import tiktoken
n_epoche = 6

def stampa_costo_training(texts):
  import tiktoken
  enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
  total_tokens=(len(enc.encode(texts)))
  print(f'Totale token:{total_tokens}')
  costo_training = (total_tokens/1000* 0.0080) * n_epoche
  print(f'Costo training GPT-3.5-Turbo in $ (OpenAI): {costo_training:.6f}')
  return costo_training

costo_training_GPT = stampa_costo_training(json_str)


Totale token:112750
Costo training GPT-3.5-Turbo in $ (OpenAI): 5.412000


Costo utilizzo del modello fine-tunato con gpt-3.5-turbo

In [ ]:
def costo_utilizzo_gpt(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
    total_tokens=(len(enc.encode(texts)))
    print(f'Totale token:{total_tokens}')
    costo_input = (total_tokens/1000* 0.0030)
    costo_output = (total_tokens/1000* 0.0060)
    costo_totale = costo_input + costo_output
    print(f'Costo utilizzo GPT-3.5-turbo in $ (OpenAI): {costo_totale:.6f}')
    return costo_totale

costo_utilizzo_gpt = costo_utilizzo_gpt(json_str)


Totale token:112750
Costo utilizzo GPT-3.5-turbo in $ (OpenAI): 1.014750


Costo stimato fine-tuning con Davinci-002

In [ ]:
import tiktoken
n_epoche = 6
def stampa_costo_embeddings(texts):
  import tiktoken
  enc = tiktoken.encoding_for_model("davinci-002")
  total_tokens=(len(enc.encode(texts)))
  print(f'Totale token:{total_tokens}')
  costo_training = (total_tokens/1000* 0.0060) * n_epoche
  print(f'Costo training Davinci-002 in $ (OpenAI): {costo_training:.6f}')
  return costo_training

costo_training_davinci = stampa_costo_embeddings(json_str)

Totale token:112750
Costo training Davinci-002 in $ (OpenAI): 4.059000


Costo utilizzo del modello fine-tunato con davinci-002

In [ ]:
def costo_utilizzo(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model("davinci-002")
    total_tokens=(len(enc.encode(texts)))
    print(f'Totale token:{total_tokens}')
    costo_input = (total_tokens/1000* 0.0120)
    costo_output = (total_tokens/1000* 0.0120)
    costo_totale = costo_input + costo_output
    print(f'Costo utilizzo Davinci-002 in $ (OpenAI): {costo_totale:.6f}')
    return costo_totale
costo_utilizzo_davinci = costo_utilizzo(json_str)


Totale token:112750
Costo utilizzo Davinci-002 in $ (OpenAI): 2.706000


Costi complessivi:

In [ ]:
costo_complessivo_gpt = costo_training_GPT +  costo_utilizzo_gpt
costo_complessivo_davinci = costo_training_davinci + costo_utilizzo_davinci

print(f'Costo complessivo GPT-3.5-turbo in $ (OpenAI): {costo_complessivo_gpt:.6f}')
print(f'Costo complessivo davinci-002 in $ (OpenAI): {costo_complessivo_davinci:.6f}')

Costo complessivo GPT-3.5-turbo in $ (OpenAI): 6.426750
Costo complessivo davinci-002 in $ (OpenAI): 5.412000


Per 90.000 token, il costo cambia come segue:

GPT-3.5-turbo:

Costo di addestramento: €0.0080 per 1K tokens x 90 = €0.72
Costo di utilizzo in input: €0.0030 per 1K tokens x 90 = €0.27
Costo di utilizzo in output: €0.0060 per 1K tokens x 90 = €0.54
davinci-002:

Costo di addestramento: €0.0060 per 1K tokens x 90 = €0.54
Costo di utilizzo in input: €0.0120 per 1K tokens x 90 = €1.08
Costo di utilizzo in output: €0.0120 per 1K tokens x 90 = €1.08
Quindi, per 90.000 token, il fine-tuning di davinci-002 risulta essere più costoso rispetto a quello di GPT-3.5-turbo.

#FINE-TUNING

In [ ]:
pip install openai==0.28

Converto il file json in un file json lista in quanto è il tipo di file accettato da OpenAI

In [ ]:
import json

def convert_json_to_jsonl(json_filename, jsonl_filename):
    with open(json_filename, 'r') as json_file:
        data = json.load(json_file)

    with open(jsonl_filename, 'w') as jsonl_file:
        for item in data:
            jsonl_file.write(json.dumps(item) + '\n')

# Example usage:
convert_json_to_jsonl('./Tesi/prova_domande_fine_tuning.json', './Tesi/solo_domande_ft_list.json')


Carico il file su OpenAI che crea un id del file che ci servirà per il fine-tuning

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

c = openai.File.create(
  file=open("./Tesi/solo_domande_ft_list.json", "rb"),
  purpose='fine-tune'
)

print(c)



{
  "object": "file",
  "id": "file-B5U4uBFyL8HbLT9dR9EySsOh",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 346097,
  "created_at": 1701083606,
  "status": "uploaded",
  "status_details": null
}


Avvio l'effettivo fine-tuning passando id file, modello e un suffisso a scelta, inoltre possiamo scegliere l'iperparametri, parto con un numero di epoche uguale a 5, se non passiamo nulla lo sceglierà in automatico.

In [ ]:
r = openai.FineTuningJob.create(training_file="file-B5U4uBFyL8HbLT9dR9EySsOh", model="davinci-002", suffix="test-Signorile",hyperparameters={"n_epochs":6})

print(r)

{
  "object": "fine_tuning.job",
  "id": "ftjob-A5MeFOFqF5YbdDDPCaXxjRmW",
  "model": "davinci-002",
  "created_at": 1701083659,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-OF04YbDrGXeJxlDm77negXuJ",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-B5U4uBFyL8HbLT9dR9EySsOh",
  "hyperparameters": {
    "n_epochs": 6,
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": null
}


Uso il jobname per controllare lo status del fine-tuning

In [ ]:
import openai

jobname = "ftjob-A5MeFOFqF5YbdDDPCaXxjRmW"

r = openai.FineTuningJob.retrieve(jobname)
e = openai.FineTuningJob.list_events(id=jobname, limit=10)
print(f"status: {r}")
print("\n\n\n")
print(f"events: {e}")

status: {
  "object": "fine_tuning.job",
  "id": "ftjob-A5MeFOFqF5YbdDDPCaXxjRmW",
  "model": "davinci-002",
  "created_at": 1701083659,
  "finished_at": 1701083988,
  "fine_tuned_model": "ft:davinci-002:links:test-signorile:8PTwTi0r",
  "organization_id": "org-OF04YbDrGXeJxlDm77negXuJ",
  "result_files": [
    "file-Kk8jh4NetdILLrj6i7Qqb26e"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-B5U4uBFyL8HbLT9dR9EySsOh",
  "hyperparameters": {
    "n_epochs": 6,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 613428,
  "error": null
}




events: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-KE7HwkK6CXca5HHYIOAVCS6H",
      "created_at": 1701083993,
      "level": "info",
      "message": "The job has successfully completed",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-OixNyijf6mbMQkvrL5u0R7Sw",
   

Costo reale fine-tuning: 3.6$